In [5]:
import glob
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)
BATCH_SIZE = 48
CROP_LEN = 112

def random_crop(im_array):
    # crop
    x_range = im_array.shape[0] - CROP_LEN
    y_range = im_array.shape[1] - CROP_LEN
    # print(x_range,y_range)
    a = np.random.randint(x_range)
    b = a + CROP_LEN
    c = np.random.randint(y_range)
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array

def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array
    

def random_manip(img,rnd):
    if rnd == 0:
        return img
    
    elif rnd == 1:
        # gamma 0.8
        new_img = skimage.exposure.adjust_gamma(img, gamma=0.8)
        return new_img
    
    elif rnd == 2:
        # gamma 1.2
        new_img = skimage.exposure.adjust_gamma(img, gamma=1.2)
        return new_img
    
    elif rnd == 3:
        # jpeg 70
        imageio.imwrite('/tmp/quality-70.jpg', img, quality=70)
        try:
            new_img = np.array(Image.open(('/tmp/quality-70.jpg')), dtype="uint8")
            os.remove('/tmp/quality-70.jpg')
            return new_img
        except:
            return img
    
    elif rnd == 4:
        # jpeg 90
        imageio.imwrite('/tmp/quality-90.jpg', img, quality=70)
        try:
            new_img = np.array(Image.open(('/tmp/quality-90.jpg')), dtype="uint8")
            os.remove('/tmp/quality-90.jpg')
            return new_img
        except:
            return img
    
    elif rnd == 5:
        # 2x of original image
        new_img = scipy.misc.imresize(img, 2.0, interp='bicubic')
        return center_crop(new_img)
    
    elif rnd == 6:
        new_img = scipy.misc.imresize(img, 1.5, interp='bicubic')
        return center_crop(new_img)
    
    elif rnd == 7:
        new_img = scipy.misc.imresize(img, 0.5, interp='bicubic')
        return new_img
    
    elif rnd == 8:
        new_img = scipy.misc.imresize(img, 0.8, interp='bicubic')
        return new_img
    
    else:
        return img

def get_img(img_path, train_flag = True):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    
    # train or valid
    if train_flag is True:
        # manip rnd
        manip_rnd = np.random.randint(16) # only half manip, 1 to 8 use manip
        #manip_rnd = 5
        #print(manip_rnd,im_array.shape)
        if manip_rnd < 7 or manip_rnd > 8 : 
            # no zoom in， 随机切出 112
            im_array = random_crop(im_array)
            # manip， 随机变化，如果放大则去中间部分
            final_img = random_manip(im_array, manip_rnd)
          
        else:
            # resize zoom out， 缩小
            im_array = random_manip(im_array, manip_rnd)
            # random crop on larger image， 随机切出 112
            final_img = random_crop(im_array)
        if final_img.shape[0]!=CROP_LEN or final_img.shape[1]!=CROP_LEN:
            print('train',manip_rnd,final_img.shape,img_path,im_array.shape)
    else:
        # center crop for valid data
        # manip rnd
        manip_rnd = np.random.randint(16)
        if manip_rnd < 7 or manip_rnd > 8 :
            # no zoom in， 取中间部分
            im_array = center_crop(im_array)
            # manip, 随机变化，如果放大则取中间部分
            final_img = random_manip(im_array, manip_rnd)
        else:
            # resize zoom out, 先缩小图片
            im_array = random_manip(im_array, manip_rnd)
            # random crop on larger image, 取中间部分
            final_img = center_crop(im_array)
        if final_img.shape[0]!=CROP_LEN or final_img.shape[1]!=CROP_LEN:
            print('valid',manip_rnd,final_img.shape,img_path,im_array.shape)
    
    final_img = final_img/255.0
    return final_img

# 读取测试数据中间 112
def get_test_img(img_path):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    final_img = center_crop(im_array)
    final_img = final_img/255.0
    return final_img

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

{'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]}


In [6]:
def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True):
    curr_idx = 0
    data_cnt = len(file_list)
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        curr_x = np.array([get_img(p,train_flag) for p in curr_fl],dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=42)


train_gen = data_gen(train_files, BATCH_SIZE, True)
valid_gen = data_gen(train_files, BATCH_SIZE, False)
train_step = train_data_cnt//BATCH_SIZE
valid_step = train_step

# test
for x,y in train_gen:
    print(x.shape,y.shape)
    print(x.dtype)
    print(y[:3])
    break
    
# test
for x,y in valid_gen:
    print(x.shape,y.shape)
    print(x.dtype)
    print(y[:3])
    break
    

(48, 112, 112, 3) (48, 10)
float32
[[0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0]]
(48, 112, 112, 3) (48, 10)
float32
[[0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0]]


In [7]:
# def model
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import backend as K

In [8]:
model_p = 'best_mobilenet_manip.h5'

if os.path.exists(model_p):
    model = load_model(model_p)
    K.set_value(model.optimizer.lr, 0.001)
    print('load',model_p)
else:
    from keras.applications.mobilenet import MobileNet
    model = MobileNet(input_shape=(CROP_LEN,CROP_LEN,3),include_top=True,weights=None,classes=10)
    print('get model')
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    
    
model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00001)
model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=100,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk,reduce_lr]
         )

get model
Epoch 1/100
57/57 [==============================] - 1045s 18s/step - loss: 2.4077 - acc: 0.1210 - val_loss: 2.3381 - val_acc: 0.1005
Epoch 2/100
57/57 [==============================] - 974s 17s/step - loss: 2.3327 - acc: 0.1499 - val_loss: 2.3728 - val_acc: 0.1005
Epoch 3/100
57/57 [==============================] - 968s 17s/step - loss: 2.3141 - acc: 0.1414 - val_loss: 2.4345 - val_acc: 0.1005
Epoch 4/100
57/57 [==============================] - 1020s 18s/step - loss: 2.2936 - acc: 0.1425 - val_loss: 2.4901 - val_acc: 0.1005
Epoch 5/100
56/57 [============================>.] - ETA: 7s - loss: 2.2761 - acc: 0.1525 Epoch 00005: val_acc did not improve

Epoch 00005: reducing learning rate to 0.0005000000237487257.
57/57 [==============================] - 981s 17s/step - loss: 2.2758 - acc: 0.1524 - val_loss: 2.4559 - val_acc: 0.1005
Epoch 6/100
57/57 [==============================] - 1073s 19s/step - loss: 2.2496 - acc: 0.1678 - val_loss: 2.4540 - val_acc: 0.1005
Epoch 7/100

OSError: cannot identify image file 'quality-70.jpg'

In [ ]:
best_model = load_model(model_p)
test_y = []
for img_p in test_files:
    tmp_x = get_test_img(img_p)
    tmp_y = best_model.predict(np.array([tmp_x]))[0]
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

In [ ]:
import pandas as pd
y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
print(df.head())
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/mobilenet_112_manip.csv',index=False)